In [3]:
import numpy as np
import pandas as pd

In [4]:
dataset = pd.read_csv("../data/training_data.csv")
dataset = pd.DataFrame(dataset)

In [5]:
dataset.columns = dataset.columns.str.strip()  # Removes leading and trailing spaces
print(dataset.isnull().sum()) # Check for missing values
print("---")
print(dataset.duplicated().sum()) # Check for duplicate values


step              0
type              0
amount            0
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64
---
0


In [6]:
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

In [7]:
dataset = dataset[['type', 'amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFraud']] 
dataset

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,CASH_IN,386385.08,4669568.85,5055953.92,506035.06,119649.98,0.0
1,CASH_IN,212458.78,234635.00,447093.78,806037.88,593579.10,0.0
2,PAYMENT,19967.60,3634.00,0.00,0.00,0.00,0.0
3,CASH_OUT,527616.51,180216.00,0.00,92157.10,619773.61,0.0
4,TRANSFER,206067.85,0.00,0.00,2131494.48,2337562.32,0.0
...,...,...,...,...,...,...,...
143920,CASH_OUT,37377.47,26397.78,0.00,753020.04,790397.50,0.0
143921,CASH_IN,184782.46,1512205.59,1696988.05,293982.59,109200.13,0.0
143922,PAYMENT,31391.33,31152.00,0.00,0.00,0.00,0.0
143923,CASH_IN,255521.98,1237508.67,1493030.65,768396.97,512874.99,0.0


In [8]:
'''
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataset is already loaded, e.g.:
# dataset = pd.read_csv("your_file.csv")

# Select independent features and dependent target
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features
Y = dataset['isFraud'] # Target

# Create a new DataFrame that includes both X and Y for plotting
data_with_target = X.copy()  # Copy X to avoid modifying it directly
data_with_target['isFraud'] = Y  # Add the target 'isFraud' to the DataFrame

# Plot pairwise scatter plots for each feature in X against Y
for col in X.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=col, y='isFraud', data=data_with_target)
    plt.title(f'Scatter plot of {col} vs isFraud')
    plt.show()
'''

'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Assuming your dataset is already loaded, e.g.:\n# dataset = pd.read_csv("your_file.csv")\n\n# Select independent features and dependent target\nX = dataset[[\'amount\',\'oldbalanceOrg\',\'newbalanceOrig\',\'oldbalanceDest\',\'newbalanceDest\']] # Features\nY = dataset[\'isFraud\'] # Target\n\n# Create a new DataFrame that includes both X and Y for plotting\ndata_with_target = X.copy()  # Copy X to avoid modifying it directly\ndata_with_target[\'isFraud\'] = Y  # Add the target \'isFraud\' to the DataFrame\n\n# Plot pairwise scatter plots for each feature in X against Y\nfor col in X.columns:\n    plt.figure(figsize=(6, 4))\n    sns.scatterplot(x=col, y=\'isFraud\', data=data_with_target)\n    plt.title(f\'Scatter plot of {col} vs isFraud\')\n    plt.show()\n'

In [9]:
X = dataset[['type', 'amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features (independent)
Y = dataset['isFraud'] # Target (dependent)
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

In [10]:
print(X.columns)
print(X[X.columns[0]].value_counts().to_dict())
print(np.unique(X[X.columns[1]]))

Index(['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest'],
      dtype='object')
{'CASH_OUT': 50733, 'PAYMENT': 48683, 'CASH_IN': 31397, 'TRANSFER': 12154, 'DEBIT': 958}
[0.00000000e+00 3.00000000e-02 8.70000000e-01 ... 3.77966840e+07
 3.87717970e+07 4.92039836e+07]


In [11]:
target_dict = Y.value_counts().to_dict()
n_labels = len(target_dict)
print(np.unique(Y))
print(target_dict)
print(n_labels)
print(type(Y)==pd.DataFrame)

[0. 1.]
{(0.0,): 143733, (1.0,): 192}
2
True


In [12]:
print(X.shape)
print(Y.shape)

(143925, 6)
(143925, 1)


In [13]:
X.iloc[1:3]


,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
1,CASH_IN,212458.78,234635.0,447093.78,806037.88,593579.1
2,PAYMENT,19967.60,3634.0,0.00,0.00,0.0


In [14]:
# Making dictionary of each value in column and how many times it occurs
dict = dataset['type'].value_counts().to_dict()
for k,v in dict.items():
    print(k,"-->",v)

len(dict)

CASH_OUT --> 50733
PAYMENT --> 48683
CASH_IN --> 31397
TRANSFER --> 12154
DEBIT --> 958


5

In [16]:
''' Entropy (Wrong)
def entropy(col,dataset):
    subset_fraud = dataset[dataset['isFraud']==1.0] # Taking only the rows with isFraud=1
    col_len = len(subset_fraud) # Length of the column
    dict_fraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset 

    ent=0 # Initialize entropy

    for k,v in dict_fraud.items():
        p = v/col_len # Probability of occurence of value in entire column
        ent -= p* np.log2(p)

    subset_nfraud = dataset[dataset['isFraud']==0.0] # Taking only the rows with isFraud=0
    col_len = len(subset_nfraud) # Length of the column
    dict_nfraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset 

    for k,v in dict_nfraud.items():
        p = v/col_len # Probability of occurence of value in entire column
        ent -= p* np.log2(p)


    return ent
    
entropy('isFraud',dataset)
'''

" Entropy (Wrong)\ndef entropy(col,dataset):\n    subset_fraud = dataset[dataset['isFraud']==1.0] # Taking only the rows with isFraud=1\n    col_len = len(subset_fraud) # Length of the column\n    dict_fraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset \n\n    ent=0 # Initialize entropy\n\n    for k,v in dict_fraud.items():\n        p = v/col_len # Probability of occurence of value in entire column\n        ent -= p* np.log2(p)\n\n    subset_nfraud = dataset[dataset['isFraud']==0.0] # Taking only the rows with isFraud=0\n    col_len = len(subset_nfraud) # Length of the column\n    dict_nfraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset \n\n    for k,v in dict_nfraud.items():\n        p = v/col_len # Probability of occurence of value in entire column\n        ent -= p* np.log2(p)\n\n\n    return ent\n    \nentropy('isFraud',dataset)\n"

In [17]:
'''Entropy Calculation for a column

def entropy_column(col,target):
    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences
    ent = 0
    for k,v in att_dict.items(): # For each unique value
        subset = dataset[dataset[col]==k] # Make subset based on unique attribute k
        length = v # Length of subset = no. of occureneces

        target_dict = subset[target].value_counts().to_dict()
        for i,j in target_dict.items():
            p = j/length # Occurences of 1s/0s divided by len of column
            ent -= p * np.log2(p)

    return ent

entropy_column('isFraud','isFraud')       

'''

"Entropy Calculation for a column\n\ndef entropy_column(col,target):\n    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences\n    ent = 0\n    for k,v in att_dict.items(): # For each unique value\n        subset = dataset[dataset[col]==k] # Make subset based on unique attribute k\n        length = v # Length of subset = no. of occureneces\n\n        target_dict = subset[target].value_counts().to_dict()\n        for i,j in target_dict.items():\n            p = j/length # Occurences of 1s/0s divided by len of column\n            ent -= p * np.log2(p)\n\n    return ent\n\nentropy_column('isFraud','isFraud')       \n\n"

In [18]:
# Find column indices and all potential splits at said indies
def potential_splits(data):
    splits = {}
    n_col = data.shape[1]
    for i in range(n_col):
        splits[i]= np.unique(data[data.columns[i]])
    
    return splits

potential_splits(X)

{0: array(['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER'],
       dtype=object),
 1: array([0.00000000e+00, 3.00000000e-02, 8.70000000e-01, ...,
        3.77966840e+07, 3.87717970e+07, 4.92039836e+07]),
 2: array([0.00000000e+00, 1.00000000e+00, 2.00000000e+00, ...,
        3.38298204e+07, 3.43357151e+07, 3.44006513e+07]),
 3: array([0.00000000e+00, 1.76000000e+00, 5.40000000e+00, ...,
        3.38313591e+07, 3.44020062e+07, 3.44519209e+07]),
 4: array([0.00000000e+00, 1.90000000e+01, 2.30000000e+01, ...,
        1.45469306e+08, 1.71939084e+08, 1.92278926e+08]),
 5: array([0.00000000e+00, 2.40500000e+01, 6.13900000e+01, ...,
        1.45494143e+08, 1.71996480e+08, 1.92307118e+08])}

In [19]:
def split_indices(x_col,att):
    left_id = np.argwhere(x_col<= att).flatten()
    right_id = np.argwhere(x_col > att).flatten()

    return left_id,right_id

left_s,right_s = split_indices(X[X.columns[1]],3.00000000e-02 )

print(X[X.columns[1]])
print(left_s)
print("---")
print(right_s)

0         386385.08
1         212458.78
2          19967.60
3         527616.51
4         206067.85
            ...    
143920     37377.47
143921    184782.46
143922     31391.33
143923    255521.98
143924     78284.35
Name: amount, Length: 143925, dtype: float64
[105307 115449]
---
[     0      1      2 ... 143922 143923 143924]


In [20]:
print(X['amount']>200000)
left = X[(X['amount']>200000)==True]
right = X[(X['amount']>200000)==False]
print(left)
print(right)

0          True
1          True
2         False
3          True
4          True
          ...  
143920    False
143921    False
143922    False
143923     True
143924    False
Name: amount, Length: 143925, dtype: bool
            type     amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0        CASH_IN  386385.08     4669568.85      5055953.92       506035.06   
1        CASH_IN  212458.78      234635.00       447093.78       806037.88   
3       CASH_OUT  527616.51      180216.00            0.00        92157.10   
4       TRANSFER  206067.85           0.00            0.00      2131494.48   
9        CASH_IN  284341.53     8133378.48      8417720.01      5702766.00   
...          ...        ...            ...             ...             ...   
143903  CASH_OUT  381436.31           0.00            0.00       423565.96   
143910   CASH_IN  253381.28       14703.00       268084.28            0.00   
143913   CASH_IN  227101.50     1267870.75      1494972.24       808471.77   
14

In [21]:
# Split at a column depending on the value of attributes in the column
def split_data(data, split_col, split_val):
    left_data = data[(data[split_col]>split_val)==False]
    right_data = data[(data[split_col]>split_val)==True]  

    return left_data, right_data

In [15]:
# Subsetting dataset
dict = dataset['isFraud'].value_counts().to_dict()
for k,v in dict.items():
    print(k,"-->",v)

subset= dataset[dataset['isFraud']==1.0]
subset.head()

0.0 --> 143733
1.0 --> 192


,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
550,TRANSFER,1735466.52,1735466.52,0.00,0.00,0.00,1.0
636,TRANSFER,316877.85,316877.85,0.00,0.00,0.00,1.0
864,CASH_OUT,78810.30,78810.30,0.00,802788.52,881598.82,1.0
870,CASH_OUT,513082.74,513082.74,0.00,32452.79,545535.53,1.0
1472,TRANSFER,2542664.27,2542664.27,2542664.27,0.00,0.00,1.0


In [48]:
# Get probabilities faster
target = dataset[dataset.columns[-1]]
counts = []
dict = target.value_counts().to_dict()
for k,v in dict.items():
    counts.append(v)
counts = np.array(counts) # Allows element-wise operations
print(counts.sum())

143925


In [50]:
# NEW calculate entropy 

def entropy(data): # Takes dataset instead of column
    target_col = dataset[dataset.columns[-1]]  # Makes last column of dataset the target column
    counts = []
    target_dict = target.value_counts().to_dict()
    for k,v in target_dict.items():
        counts.append(v)
    counts = np.array(counts)

    p = counts / counts.sum() # Does calculation at once, so faster
    ent = -np.sum(p * np.log2(p))

    return ent

ent_dataset(dataset)

np.float64(0.014663273789461197)

In [52]:
# Entropy calculation for an attribute in a column
def entropy_attribute(col,val,target):
    subset = dataset[dataset[col]==val] # Make subset based on attribute 
    
    ent = 0 # Initialize entropy value

    target_dict = subset[target].value_counts().to_dict()
    for k,v in target_dict.items():
        p = v/len(subset) # Occurences of 1s/0s divided by len of column
        ent -= p * np.log2(p) # Entropy of attribute

    return ent

entropy_attribute('isFraud',1.0,'isFraud')

np.float64(0.0)

In [51]:
# Entropy of parent
'''
def entropy_dataset(target):
    target_dict = dataset[target].value_counts().to_dict()
    ent = 0
    for k,v in target_dict.items():
        p = v/len(dataset)
        ent -= p * np.log2(p)
    
    return ent

entropy_dataset('isFraud')
'''

np.float64(0.014663273789461197)

In [ ]:
# Information Gain Calculation
def information_gain(col,target):
    len_data = len(dataset) # Length of dataset
    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences

    ent_data = entropy_dataset(target) # Calculate entropy of dataset
    ent_att = 0 # Initalize entropy of attributes of a column

    for k,v in att_dict.items():
        p = v / len_data # No. of occurences of attributes / length of dataset
        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column
    
    info_gain = ent_data + ent_att # Calculate information gain

    return info_gain

information_gain('type','isFraud')


"\ndef information_gain(col,target):\n    len_data = len(dataset) # Length of dataset\n    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences\n\n    ent_data = entropy_dataset(target) # Calculate entropy of dataset\n    ent_att = 0 # Initalize entropy of attributes of a column\n\n    for k,v in att_dict.items():\n        p = v / len_data # No. of occurences of attributes / length of dataset\n        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column\n    \n    info_gain = ent_data + ent_att # Calculate information gain\n\n    return info_gain\n\ninformation_gain('type','isFraud')\n"

In [ ]:
# Nodes
class node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature # Column
        self.threshold = threshold # Attribute in column
        self.left = left # Left child
        self.right = right # Right child
        self.value = None # Label of node (for leaf node only)

    def is_leaf(self): # Check if node is leaf node
        if self.value != None:
            return self.value

In [ ]:
# Decision Tree
class decision_tree:
    def __init__(self, min_samples=5, max_depth=100, n_features=None):
        self.min_samples = min_samples # Minimum number of samples at a node
        self.max_depth = max_depth # Max height of tree
        self.n_features = n_features # Number of feautures to use to make tree
        self.root = None # Root node

    def fit(self,x,y):
        if self.n_features == None: # To ensure no. of features(columns) <= no. of actual features
            self.n_features = x.shape[1]
        else:
            self.n_features = min(x.shape[1],self.n_features)

        self.root = self.build_tree(x,y) # Build tree starting from root

    depth = 0 # Initialize depth of tree

    def build_tree(self,x,y,depth):
        # Check if stopping criteria reached
        # If not find the best split
        # Create child nodes based on split condition

        n_samples = x.shape[0] # No. of sample points = no. of rows
        n_feats = x.shape[1] # No. of features = no. of columns
        n_labels = len(np.unique(y)) # No. of unique labels

        if depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples: # Stopping criteria
            leaf_label = self.majority_label(y) # Label of leaf should be the majority label in the node
            return node(value=leaf_label) # Creates a leaf node and exits build_tree function

        x_col = x.columns # Make list of feaures(columns) of X
        feat_indices = np.random.choice(n_feats, self.n_features, replace =False) # Selecting random indices of features to be used
        feat_list = [] # Make a list of features to be used
        for i in feat_indices:
            feat_list.append(x_col[i]) # Add features from X


        best_feature, best_threshold = self.best_split(x,y,feat_list) # Finding the best feature and threshold to split at

        

    def best_split(self,x,y,feat_list):
        best_gain = -1 # Initialize best information gain
        split_feat, split_threshold = None, None

        for i in feat_list: # For each column of X
            x_dict = x[x.columns[i]].value_counts().to_dict() # Make dictionary out of attributes of column i of X
            
            for k,v in x_dict.items():
                info_gain = information_gain(x.columns[i],y.columns[0])

                if info_gain>best_gain:
                    best_gain = info_gain # Update info gain
                    split_feat = i # Make splitting feature = to current feature in feat_list 
                    split_threshold = k # Make splitting threshold = the attribute in column 
        
        return split_feat, split_threshold
            

    def majority_label(self,y):
        if type(y) != pd.DataFrame:
            y = pd.DataFrame(y)
        y_dict = y.value_counts().to_dict()

        max_key,max_val = "",0
        for k,v in y_dict.items():
            if v>max_val:
                max_key,max_val=k,v
        return max_key
        


        
